# Google Spreadsheet Interface

## Open Data Science Initiative

### 14th August 2014 Neil Lawrence

### Updated 17th October 2014 Neil Lawrence
### Updated 19th October 2015 Neil Lawrence

As part of the distribution of information to area chairs in NIPS 2014 we used Google docs to provide summary spreadsheets. Review information is processed in `pandas` so this rough interface is designed to allow uploading and downloading of information from google spreadsheets as pandas data frames.

In [1]:
import pods.google
import pandas as pd

Next you'll need to make use of two gmail user IDs for this example.


In [2]:
user1 = 'user1@gmail.com'
user2 = 'user2@sheffield.ac.uk'

First let's create a simple data frame for placing in a spreadsheet.

In [5]:
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])} 

df = pd.DataFrame(d)

In [6]:
df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


### Creating a Google Sheet
Now we create the google document and write our data frame, `df`, to the spreadsheet.

In [7]:
sheet = pods.google.sheet()

The spreadsheet is uniquely defined by its resource `id`, which can be recovered from the spreadsheet as follows:

In [8]:
print(sheet.resource._id)

1U-by6mHRllDMinl4zYvdvxH_2wjNmnl97pHAPAjKSHc


The spreadsheet can be shared (by default with 'edit' access) to other gmail users.

In [9]:
sheet.share(user1)

Response for request_id (batch0):
{'kind': 'drive#permission', 'photoLink': 'https://lh5.googleusercontent.com/-MF4POUSmW-w/AAAAAAAAAAI/AAAAAAAAI3Y/MbeFfBJMTLc/s64/photo.jpg', 'role': 'writer', 'emailAddress': 'lawrennd@gmail.com', 'id': '12822106802021894772', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1U-by6mHRllDMinl4zYvdvxH_2wjNmnl97pHAPAjKSHc/permissions/12822106802021894772', 'etag': '"uZmUWEX85gMuWwYLS4M-3KqR9nM/Qh7A9R1tYkmOHz-H-e4BA2E3bg8"', 'domain': 'gmail.com', 'type': 'user', 'name': 'Neil Lawrence', 'deleted': False}


In [10]:
sheet.share_delete(user1)

To share with 'view' access you pass an extra argument, `share_type='reader'`. You can also send a notification (by default there is no mail sent).

In [11]:
sheet.share(user1)
sheet.share_modify(user1,share_type='reader',send_notifications=True)

Response for request_id (batch0):
{'kind': 'drive#permission', 'photoLink': 'https://lh5.googleusercontent.com/-MF4POUSmW-w/AAAAAAAAAAI/AAAAAAAAI3Y/MbeFfBJMTLc/s64/photo.jpg', 'role': 'writer', 'emailAddress': 'lawrennd@gmail.com', 'id': '12822106802021894772', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1U-by6mHRllDMinl4zYvdvxH_2wjNmnl97pHAPAjKSHc/permissions/12822106802021894772', 'etag': '"uZmUWEX85gMuWwYLS4M-3KqR9nM/Qh7A9R1tYkmOHz-H-e4BA2E3bg8"', 'domain': 'gmail.com', 'type': 'user', 'name': 'Neil Lawrence', 'deleted': False}


The spreadsheet's title can also be renamed.

In [12]:
sheet.set_title('New Title')

And the new title exhibited.

In [13]:
sheet.get_title()

'New Title'

Or it can be deleted.

In [14]:
sheet.resource.delete()

### Loading an Existing Sheet

Now, for demonstration pruposes, here's the key of 'one we made earlier'. You can open an existing spreadsheet by providing the key as follows

In [15]:
import pods
resource = pods.google.resource(id='1nnUOpV30Jo9ISYPtkqSWbTLzymo5EBxA9056gKdLh1Q')
sheet = pods.google.sheet(resource=resource)

In [16]:
df2 = sheet.read()

In [17]:
df2

,one,two
index,,
a,Sheffield,1
b,Open,2
c,Data,3
d,Science,4


For this google doc, now it's created we've published it to the web. Quoting from [the developer's site](https://developers.google.com/google-apps/spreadsheets/)

"Publishing a spreadsheet to the web can only be done from the Google sheet's user interface. To start publishing a spreadsheet to the web, select `File > Publish` to the web from the sheet's user interface, and then click the Start Publishing button."

Once the spreadsheet is published you can have people viewing it publicly as follows.

In [18]:
sheet.share([user1, user2])

Response for request_id (batch0):
{'kind': 'drive#permission', 'photoLink': 'https://lh5.googleusercontent.com/-MF4POUSmW-w/AAAAAAAAAAI/AAAAAAAAI3Y/MbeFfBJMTLc/s64/photo.jpg', 'role': 'writer', 'emailAddress': 'lawrennd@gmail.com', 'id': '12822106802021894772', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1nnUOpV30Jo9ISYPtkqSWbTLzymo5EBxA9056gKdLh1Q/permissions/12822106802021894772', 'etag': '"uZmUWEX85gMuWwYLS4M-3KqR9nM/XEjw9mYU9YbsbiEpWZA6-y3UZ6o"', 'domain': 'gmail.com', 'type': 'user', 'name': 'Neil Lawrence', 'deleted': False}
Response for request_id (batch1):
{'kind': 'drive#permission', 'photoLink': 'https://lh3.googleusercontent.com/-zWT7RVjCwrs/AAAAAAAAAAI/AAAAAAAAAB8/3su2t0gWzbY/s64/photo.jpg', 'role': 'writer', 'emailAddress': 'n.lawrence@sheffield.ac.uk', 'id': '10282882308563513722', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1nnUOpV30Jo9ISYPtkqSWbTLzymo5EBxA9056gKdLh1Q/permissions/10282882308563513722', 'etag': '"uZmUWEX85gMuWwYLS4M-3KqR9nM/g8x8Z0iHC

Or you can modify the access rights they have to the document.

In [19]:
sheet.share_modify(user2, 'writer')

Or you can remove them from the sharing list completely.

In [20]:
sheet.share_delete(user1)

To see who the document is shared with you can use the `share_list()` method.

In [21]:
sheet.share_list()

[('754462758110-tgc1l41ictkaj9m4t82d3okm04k1pn7o@developer.gserviceaccount.com',
  'owner'),
 ('n.lawrence@sheffield.ac.uk', 'writer')]

### Displaying the Sheet

You can display the sheet in IPython, this action downloads the sheet and shows the associated `pandas` dataframe.

In [22]:
from IPython.display import display
display(sheet)


,one,two
index,,
a,Sheffield,1
b,Open,2
c,Data,3
d,Science,4


Or if the spreadsheet is published you can set `sheet.published = True` and get a representation of the spreadsheet embedded in the notebook.

## Reading and Modifying the Google Sheet

To read the data frame back from the spreadsheet you simply use the `read` method.

In [23]:
df2 = sheet.read()
df2

,one,two
index,,
a,Sheffield,1
b,Open,2
c,Data,3
d,Science,4


We can now update the spreadsheet, by modifying the data frame, and then requesting an update. The update command looks only for entries that have changed.

In [24]:
df2['one']['a'] = 2.3
sheet.update(df2)
sheet

/Users/neil/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,one,two
index,,
a,2.3,1
b,Open,2
c,Data,3
d,Science,4


You should see the spreadsheet has changed. You can also delete rows.

In [25]:
sheet.update(df2.drop('a'))
display(sheet)

,one,two
index,,
b,Open,2
c,Data,3
d,Science,4


Or add them in again. Here we update back to the original data frame (`df`). Note when you look at the google doc that the row is added at the end of the spreadsheet. This is so that any sorting of other rows in the spreadsheet is preserved.

In [26]:
sheet.update(df)

Let's open the sheet again with a new interface.

In [27]:
sheet2 = pods.google.sheet(resource=sheet.resource)
df4 = sheet2.read()
print(df4)

       one  two
index          
b      2.0    2
c      3.0    3
d      NaN    4
a      1.0    1


Note that because we deleted the row indexed by `'b'` and then updated the spreadsheet, the row of `b` was added back at the end of the spreadsheet.

In [28]:
sheet2.update(df4.drop('c'))

In [29]:
display(sheet2)

,one,two
index,,
b,2.0,2
d,NaN,4
a,1.0,1
